In [10]:
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import accuracy_score

#read data
train=pd.read_csv('E:/data/train_titanic.csv')
test=pd.read_csv("E:/data/test_titanic.csv")
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [12]:
def clean_data(titanic):
    #imputate missing values with median
    titanic["Age"]=titanic["Age"].fillna(titanic["Age"].median())
    #identify children
    titanic["child"]=titanic["Age"].apply(lambda x:1 if x<15 else 0)
    #sex
    titanic["sex"]=titanic["Sex"].apply(lambda x:1 if x=="male" else 0)
    titanic["Embarked"]=titanic["Embarked"].fillna("S")
    def getEmbark(Embarked):
        if Embarked=="S":
            return 1
        elif Embarked=="C":
            return 2
        else:
            return 3
    titanic["embark"] = titanic["Embarked"].apply(getEmbark)
    #familysize
    titanic["fimalysize"] = titanic["SibSp"] + titanic["Parch"] + 1
    # cabin
    def getCabin(cabin):
        if cabin == "N":
            return 0
        else:
            return 1
    titanic["cabin"] = titanic["Cabin"].apply(getCabin)
    # name
    def getName(name):
        if "Mr" in str(name):
            return 1
        elif "Mrs" in str(name):
            return 2
        else:
            return 0
    titanic["name"] = titanic["Name"].apply(getName)
    titanic["Fare"] = titanic["Fare"].fillna(titanic["Fare"].median())

    return titanic
# 对数据进行清洗
train_data = clean_data(train)
test_data = clean_data(test)

features = ["Pclass", "sex", "child", "fimalysize", "Fare", "embark", "cabin"]
#构造模型
clf =XGBClassifier(learning_rate=0.1, max_depth=2, silent=True, objective='binary:logistic')
#kfold
# 设置boosting迭代计算次数
param_test = {
    'n_estimators': range(30, 50, 2),
    'max_depth': range(2, 7, 1)
}
grid_search = GridSearchCV(estimator = clf, param_grid = param_test, scoring='accuracy', cv=5)
grid_search.fit(train[features], train["Survived"])
grid_search.grid_scores_, grid_search.best_params_, grid_search.best_score_


([mean: 0.81594, std: 0.00673, params: {'n_estimators': 30, 'max_depth': 2},
  mean: 0.81930, std: 0.00916, params: {'n_estimators': 32, 'max_depth': 2},
  mean: 0.82267, std: 0.00978, params: {'n_estimators': 34, 'max_depth': 2},
  mean: 0.82043, std: 0.01423, params: {'n_estimators': 36, 'max_depth': 2},
  mean: 0.82267, std: 0.01585, params: {'n_estimators': 38, 'max_depth': 2},
  mean: 0.82604, std: 0.01800, params: {'n_estimators': 40, 'max_depth': 2},
  mean: 0.82604, std: 0.01800, params: {'n_estimators': 42, 'max_depth': 2},
  mean: 0.82379, std: 0.01629, params: {'n_estimators': 44, 'max_depth': 2},
  mean: 0.82379, std: 0.01629, params: {'n_estimators': 46, 'max_depth': 2},
  mean: 0.82267, std: 0.01545, params: {'n_estimators': 48, 'max_depth': 2},
  mean: 0.82043, std: 0.01642, params: {'n_estimators': 30, 'max_depth': 3},
  mean: 0.81930, std: 0.01690, params: {'n_estimators': 32, 'max_depth': 3},
  mean: 0.81818, std: 0.01863, params: {'n_estimators': 34, 'max_depth': 3},